# Apriori Algorithm

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [14]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.frequent_patterns import apriori,association_rules
import sklearn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df = pd.read_excel("/content/gdrive/My Drive/ML_practical/Apriori algorithm/online_retail_II.xlsx")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
df.tail()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom


In [16]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(525461, 8)

In [17]:
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [20]:
df.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [21]:
df.Country.unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object)

In [23]:
from tables import Description
df['Description'] = df['Description'].str.strip()

df.dropna(axis = 0,subset = ['Invoice'],inplace = True)
df['Invoice'] = df['Invoice'].astype('str')

df = df[~df['Invoice'].str.contains('C')]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
#split the data according to the region of transaction
basket1_France = (df[df['Country'] == 'France']
            .groupby(['Invoice','Description'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('Invoice'))

basket1_UK = (df[df['Country'] == 'United Knigdom']
          .groupby(['Invoice','Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

basket1_Por = (df[df['Country'] == 'Portugal']
          .groupby(['Invoice','Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

basket1_Sweden = (df[df['Country'] == 'Sweden']
            .groupby(['Invoice','Description'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('Invoice'))

print(basket1_France)

Description  10 COLOUR SPACEBOY PEN  12 IVORY ROSE PEG PLACE SETTINGS  \
Invoice                                                                 
489439                          0.0                               0.0   
489557                          0.0                               0.0   
489883                          0.0                               0.0   
490139                          0.0                               0.0   
490152                          0.0                               0.0   
...                             ...                               ...   
537693                          0.0                               0.0   
537897                          0.0                               0.0   
537967                          0.0                               0.0   
538008                          0.0                               0.0   
538093                          0.0                               0.0   

Description  12 MESSAGE CARDS WITH ENVELOPES  12 P

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
#hot encodding
def hot_encode1(P):
  if(P<= 0):
    return 0
  if(P>= 1):
    return 1

basket1_encoded = basket1_France.applymap(hot_encode1)
basket1_France  = basket1_encoded
print(basket1_France)

basket1_encoded = basket1_UK.applymap(hot_encode1)
basket1_UK  = basket1_encoded
print(basket1_UK)

basket1_encoded = basket1_Por.applymap(hot_encode1)
basket1_Por  = basket1_encoded
print(basket1_Por)

basket1_encoded = basket1_Sweden.applymap(hot_encode1)
basket1_Sweden  = basket1_encoded
print(basket1_Sweden)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Description  10 COLOUR SPACEBOY PEN  12 IVORY ROSE PEG PLACE SETTINGS  \
Invoice                                                                 
489439                            0                                 0   
489557                            0                                 0   
489883                            0                                 0   
490139                            0                                 0   
490152                            0                                 0   
...                             ...                               ...   
537693                            0                                 0   
537897                            0                                 0   
537967                            0                                 0   
538008                            0                                 0   
538093                            0                                 0   

Description  12 MESSAGE CARDS WITH ENVELOPES  12 P

In [32]:
#Build the models and analyse the results

#a)France

frq_items1 = apriori(basket1_France , min_support = 0.05 , use_colnames = True)

#collect the inferrd rules in dataframe
rules1 = association_rules (frq_items1 , metric='lift', min_threshold=1)
rules1 = rules1.sort_values(['confidence','lift'],ascending = [False,False])
print(rules1.head)

<bound method NDFrame.head of                                            antecedents  \
415  (POSTAGE, PLASTERS IN TIN CIRCUS PARADE, PLAST...   
303  (PLASTERS IN TIN STRONGMAN, PLASTERS IN TIN SP...   
27                          (COFFEE MUG APPLES DESIGN)   
99                                  (PINK  SPOTTY CUP)   
131                         (RED RETROSPOT CAKE STAND)   
..                                                 ...   
170                                          (POSTAGE)   
393                                          (POSTAGE)   
227                                          (POSTAGE)   
251                                          (POSTAGE)   
351                                          (POSTAGE)   

                                           consequents  antecedent support  \
415                 (PLASTERS IN TIN WOODLAND ANIMALS)            0.058091   
303                    (PLASTERS IN TIN CIRCUS PARADE)            0.053942   
27                                     

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [33]:
#https://www.javatpoint.com/implementing-apriori-algorithm-in-python

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
